In [293]:
using LinearAlgebra, SparseArrays
import gmsh
gmsh.initialize()

include("FEM.jl")
using .FEM

In [294]:
# Adatok
E = 2e5
ν = 0.3
thickness = 1
height = 100
base = 100
elemSize = 22 #22

approxOrder = 2
internalNodes = true
quadElements = true

problem = FEM.Problem("rectangle", type="PlaneStress", E=E, ν=ν, b=thickness)

Main.FEM.Problem("rectangle", "PlaneStress", 200000.0, 0.3, 7.85e-9, 1.0)

In [295]:
# Geometria megrajzolása
gmsh.model.add(problem.name)

p1 = gmsh.model.occ.addPoint(0, 0, 0)
p2 = gmsh.model.occ.addPoint(base, 0, 0)
#p23 = gmsh.model.occ.addPoint(base, height / 2, 0)
p3 = gmsh.model.occ.addPoint(base, height, 0)
p4 = gmsh.model.occ.addPoint(0, height, 0)

l1 = gmsh.model.occ.addLine(p1, p2)
l2 = gmsh.model.occ.addLine(p2, p3)
#l23 = gmsh.model.occ.addLine(p23, p3)
l3 = gmsh.model.occ.addLine(p3, p4)
l4 = gmsh.model.occ.addLine(p4, p1)

cl1 = gmsh.model.occ.addCurveLoop([l1, l2, l3, l4])

l5 = gmsh.model.occ.addCircle(base / 2, height / 2, 0, min(base, height) / 4)
cl2 = gmsh.model.occ.addCurveLoop([l5])

sf1 = gmsh.model.occ.addPlaneSurface([cl1, cl2])
#display(sf1)

#cp1 = gmsh.model.occ.addPoint(base / 2, height / 2, 0)
#cl1 = gmsh.model.occ.addLine(p1, p3)
#cl2 = gmsh.model.occ.addLine(p2, p4)

#sf2 = gmsh.model.occ.fragment([(2, sf1)], [(1, cl1)])
#gmsh.model.occ.removeAllDuplicates()

#p5 = gmsh.model.occ.addPoint(0, height / 2, 0)
#p6 = gmsh.model.occ.addPoint(base, height / 2, 0)
#l5 = gmsh.model.occ.addLine(p5, p6)

gmsh.model.occ.synchronize()
#sf2

In [296]:
p5 = gmsh.model.occ.addPoint(0, height / 2, 0)
p6 = gmsh.model.occ.addPoint(base / 3, height / 2, 0)
p7 = gmsh.model.occ.addPoint(3base / 4, height / 2, 0)
p8 = gmsh.model.occ.addPoint(base, height / 2, 0)
l6 = gmsh.model.occ.addLine(p5, p6)
l7 = gmsh.model.occ.addLine(p7, p8)
sf2 = gmsh.model.occ.fragment([(2, sf1)], [(1, l6), (1, l7)])
gmsh.model.occ.synchronize()
sf2

(Tuple{Int32, Int32}[(1, 7), (2, 1), (2, 2), (1, 17)], Vector{Tuple{Int32, Int32}}[[(2, 1), (2, 2)], [(1, 9), (1, 17)], [(1, 7)]])

In [297]:
ee = 1
innerline = gmsh.model.getEntitiesInBoundingBox(base / 4 - ee, height / 2 - ee, 0 - ee, 3base / 4 + ee, height / 2 + ee, 0 + ee, 1)

1-element Vector{Tuple{Int32, Int32}}:
 (1, 17)

In [298]:
gmsh.model.occ.remove(innerline, true)
gmsh.model.occ.synchronize()

In [299]:
supp = gmsh.model.getEntitiesInBoundingBox(0 - ee, 0 - ee, 0 - ee, 0 + ee, height + ee, 0 + ee, 1)
load = gmsh.model.getEntitiesInBoundingBox(0 - ee, height - ee, 0 - ee, base + ee, height + ee, 0 + ee, 1)

1-element Vector{Tuple{Int32, Int32}}:
 (1, 14)

In [300]:
supp1 = FEM.displacementConstraintOnLines([supp[i][2] for i in 1:length(supp)], ux=0, uy=0)
load1 = FEM.tractionOnLines([load[i][2] for i in 1:length(load)], fx=0, fy=1, thickness=thickness)

("load", 0, 1, 1)

In [301]:
FEM.generateMesh(problem, sf1, elemSize, approxOrder=approxOrder, algorithm=8, quadrangle=quadElements, internalNodes=internalNodes)

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 7 (Line)
Info    : [ 10%] Meshing curve 8 (Line)
Info    : [ 20%] Meshing curve 9 (Line)
Info    : [ 30%] Meshing curve 10 (Circle)
Info    : [ 40%] Meshing curve 11 (Line)
Info    : [ 50%] Meshing curve 12 (Line)
Info    : [ 60%] Meshing curve 13 (Line)
Info    : [ 70%] Meshing curve 14 (Line)
Info    : [ 80%] Meshing curve 15 (Line)
Info    : [ 90%] Meshing curve 16 (Circle)
Info    : Done meshing 1D (Wall 0.000871078s, CPU 0.000871s)
Info    : 32 nodes 42 elements
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 1 (Plane, Frontal-Delaunay for Quads)
Info    : [ 50%] Meshing surface 2 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.00210739s, CPU 0.002017s)
Info    : 48 nodes 108 elements
Info    : Recombining 2D mesh...
Info    : Blossom recombination completed (Wall 0.000205859s, CPU 0.000193s): 12 quads, 3 triangles, 0 invalid quads, 0 quads with Q < 0.1, avg Q = 0.833016, min Q = 0.665865
Info    : Blossom

In [302]:
gmsh.model.mesh.removeDuplicateNodes()
gmsh.model.mesh.renumberNodes()

Info    : Removing duplicate mesh nodes...
Info    : Found 0 duplicate nodes 
Info    : No duplicate nodes found


In [303]:
#FEM.revertTriangles(problem)

In [304]:
gmsh.fltk.run()

-------------------------------------------------------
Version       : 4.12.0
License       : GNU General Public License
Build OS      : Linux64-sdk
Build date    : 20231221
Build host    : gmsh.info
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blas[petsc] Blossom Cgns DIntegration Dlopen DomHex Eigen[contrib] Fltk Gmm[contrib] Hxt Jpeg Kbipack Lapack[petsc] LinuxJoystick MathEx[contrib] Med Mesh Metis[contrib] Mmg Mpeg Netgen ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom PETSc Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR Voro++[contrib] WinslowUntangler Zlib
FLTK version  : 1.4.0
PETSc version : 3.14.4 (real arithmtic)
OCC version   : 7.7.2
MED version   : 4.1.0
Packaged by   : geuzaine
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------


In [305]:
# Néhány láthatósági beállítás 0:kikapcs. 1:bekapcs.
gmsh.option.setNumber("Geometry.CurveLabels", 0)
gmsh.option.setNumber("Geometry.Points", 0)
gmsh.option.setNumber("Geometry.Curves", 0)
gmsh.option.setNumber("Mesh.Points", 0)
gmsh.option.setNumber("Mesh.Lines", 0)
gmsh.option.setNumber("Mesh.SurfaceEdges", 0)
gmsh.option.setNumber("Mesh.NodeLabels", 0)
gmsh.option.setNumber("Mesh.LineLabels", 0)
gmsh.option.setNumber("Mesh.SurfaceLabels", 0)

In [306]:
K = FEM.stiffnessMatrixPlaneStress(problem)

324×324 SparseMatrixCSC{Float64, Int64} with 8784 stored entries:
⎡⣿⣿⡟⠻⠲⠐⠀⢨⠀⠀⠀⡇⡇⠰⠶⠀⣽⠀⠀⠀⢸⣯⣥⠀⠀⠀⠰⠇⠀⠀⠀⠀⠲⠀⠈⠶⠊⠀⠞⠁⎤
⎢⣿⡉⢿⣷⣿⡸⠀⢰⠀⠀⠀⠁⠁⢀⣀⠀⠉⠀⠀⠀⠈⠉⠉⠀⠀⠗⣸⣆⡸⠀⡀⢀⢿⠀⢲⣀⣷⠂⣿⡆⎥
⎢⢘⠂⣛⡻⣿⣿⠀⢀⡀⡀⠀⠀⠀⠐⠒⡀⣀⠀⠀⠀⠀⢀⠀⣀⢀⣀⣿⡯⣿⡀⠇⠨⡓⠀⣀⡿⣿⡀⣛⢅⎥
⎢⡀⣀⢀⣀⠀⢀⠿⢇⡸⡸⠏⠎⠁⠷⠎⠉⣸⠏⠀⠿⠆⢀⡀⣸⢇⢹⠇⡿⠀⠀⠈⠀⠀⠸⣇⡀⢀⠾⢰⡉⎥
⎢⠀⠀⠀⠀⠀⠨⣒⡪⢿⢗⠀⢐⠐⠀⠀⠀⢽⣗⢗⣒⠀⠨⢑⣿⡺⠈⠀⠁⠀⠀⠀⠀⠀⠀⠉⠁⠀⠀⠀⠀⎥
⎢⠤⠤⠄⠀⠀⠀⡫⠅⢀⢀⣟⣽⣧⡃⠀⠀⢸⣟⣘⣻⢧⠇⢀⡄⣤⡄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⢉⡉⠁⢀⢀⠀⢥⡄⠐⠀⠭⠻⠿⣧⣄⠀⠘⠿⠂⠭⠘⠁⠐⠃⠛⣧⣤⣤⠀⢠⠀⠀⢠⣤⠀⣀⠀⣤⣤⠀⎥
⎢⠘⠃⠀⠘⠘⠠⡎⠁⠀⠀⠀⠀⠀⠙⢻⣶⡆⠀⠀⠀⠀⠀⠀⠀⠀⣿⣾⢻⡦⡾⣶⣶⠘⠉⡦⠟⠠⢽⠋⣶⎥
⎢⠓⠛⠃⠀⠀⠘⡶⠞⢷⢷⣶⢶⣶⡄⠈⠉⠿⣧⢷⣶⠾⠞⢿⡿⠾⠏⠉⠈⠁⠁⠀⠁⠀⠀⠈⠁⠈⠁⠀⠈⎥
⎢⠀⠀⠀⠀⠀⠀⣤⡄⢹⢱⣶⣸⡌⡄⠀⠀⢹⣷⠱⣦⡆⠆⠙⢫⡍⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⡶⣶⡆⠀⠀⢀⠈⢁⡀⡀⠭⠗⠖⠀⠀⠀⣺⠇⠨⠍⢵⢗⣒⣂⢒⠂⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠁⠛⠃⠀⠀⢠⣀⣨⣵⣴⠀⠴⠴⠀⠀⠀⣿⡷⡷⣀⠸⢸⡿⣯⣶⠄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⢤⠄⠀⢰⣍⣑⡚⠊⠀⠿⠿⣤⣤⣤⡾⠇⠃⠉⠸⠐⠘⠟⠿⣧⣤⣤⠠⣤⣤⠀⣤⣤⣤⡄⢠⣤⡄⣤⎥
⎢⠴⠆⠲⢾⡿⡿⣭⡥⠄⠀⠀⠀⠀⣿⣾⣛⡃⠀⠀⠀⠀⠀⠀⠀⠀⣿⣿⣿⡛⡻⡿⣋⠶⣯⡥⢿⠿⣭⣷⡍⎥
⎢⠀⠀⠒⠊⠛⠻⠀⠀⠀⠀⠀⠀⠀⣀⣨⡯⠅⠀⠀⠀⠀⠀⠀⠀⠀⣦⣿⡨⢿⣷⣀⠍⢒⡀⠠⠌⠻⠄⠀⠡⎥
⎢⠀⠀⠀⢈⡉⡁⠂⠀⠀⠀⠀⠀⠀⠀⢸⣿⠄⠀⠀⠀⠀⠀⠀⠀⠀⠛⡿⢫⡄⠜⡟⣭⠀⠀⠂⢉⠁⠐⠀⡒⎥
⎢⠘⠂⠛⠓⠙⠈⣀⡀⠀⠀⠀⠀⠀⣶⡖⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣿⡼⣧⠘⠰⠀⠀⠻⣦⡉⠒⠉⣥⣏⠀⎥
⎢⢢⡄⠘⢲⣤⡼⠉⠹⠇⠀⠀⠀⠀⢠⣬⠏⠆⠀⠀⠀⠀⠀⠀⠀⠀⠿⣥⣏⡀⠆⡌⢀⢣⠈⠿⣧⡼⠏⡔⡻⎥
⎢⠊⠀⠹⠛⠛⠻⣠⡔⠀⠀⠀⠀⠀⣤⣄⣆⠆⠀⠀⠀⠀⠀⠀⠀⠀⣶⡟⣧⠛⠆⢁⠀⠇⣤⡶⠏⠻⣦⣽⣲⎥
⎣⠞⠁⠻⠿⠟⢜⡔⠲⠀⠀⠀⠀⠀⠛⢫⣤⡀⠀⠀⠀⠀⠀⠀⠀⠀⣭⡝⠿⠄⡀⢠⠠⠋⠙⣴⡩⢳⣻⠻⣦⎦

In [307]:
K, f = FEM.applyBoundaryConditions!(problem, K, [supp1], [load1])

(sparse([1, 2, 3, 4, 5, 6, 7, 8, 29, 30  …  209, 210, 237, 238, 285, 286, 301, 302, 323, 324], [1, 2, 3, 4, 5, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34,

In [308]:
#det(K)

In [309]:
q = FEM.solveDisplacement(K, f)

324×1 Matrix{Float64}:
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
 -0.0007917832559534723
  0.0033893938953249683
  0.0
  0.0
  0.0
  0.0
  0.0
  ⋮
 -0.00016196271669931462
  0.0002036785505269287
 -0.0003039757448646414
  0.003006269189994094
  2.5937398603410267e-5
  0.00010492950606503738
 -0.0006837114535935956
  0.0022209341206345912
 -0.000245136557188368
  0.0028920870788818635
  0.0001327278953209777
  0.0003899671724426412

In [310]:
S = FEM.solveStressPlaneStress(problem, q)

([[-1.1269459154863233, -0.19944216755407096, 0.0, -0.19944216755407096, -0.10181394668311478, 0.0, 0.0, 0.0, 0.0, 0.38710475824011403  …  0.0, -0.8201460864368124, 0.09748274929899313, 0.0, 0.09748274929899313, -0.027613902735577484, 0.0, 0.0, 0.0, 0.0], [-2.722774574353191, 1.9301438435136533, 0.0, 1.9301438435136533, -1.653848536475212, 0.0, 0.0, 0.0, 0.0, 1.0627573974245585  …  0.0, -2.6930081371968146, 1.5951782361776576, 0.0, 1.5951782361776576, -1.19753209544259, 0.0, 0.0, 0.0, 0.0], [1.8396495931957695, 1.602889859911155, 0.0, 1.602889859911155, 2.1439786179270106, 0.0, 0.0, 0.0, 0.0, 0.1970461080669391  …  0.0, 1.140945608099964, 0.838917008530123, 0.0, 0.838917008530123, 1.4806961539078136, 0.0, 0.0, 0.0, 0.0], [-1.0032038211579701, 1.7993978906617984, 0.0, 1.7993978906617984, 1.5594908226118094, 0.0, 0.0, 0.0, 0.0, -0.6805958222426665  …  0.0, -1.3717797456252492, 1.0726693622091739, 0.0, 1.0726693622091739, 1.248813776328185, 0.0, 0.0, 0.0, 0.0], [0.10361758359437181, -0.10

In [311]:
gmsh.option.setNumber("View.IntervalsType", 3)
gmsh.option.setNumber("View.VectorType", 5)
gmsh.option.setString("View.Format", "%.6g")

In [312]:
FEM.revertTriangles(problem)

In [313]:
u = FEM.showResultUvec(problem, q, name="uvec", visible=false)
ux = FEM.showResultUX(problem, q, name="ux", visible=false)
uy = FEM.showResultUY(problem, q, name="uy", visible=false)
s = FEM.showResultS(problem, S, name="σ", visible=false, smooth=false)
sx = FEM.showResultSX(problem, S, name="σx", visible=false, smooth=false)
sy = FEM.showResultSY(problem, S, name="σy", visible=true, smooth=false)
sxy = FEM.showResultSXY(problem, S, name="τxy", visible=false, smooth=false)

6

In [314]:
pp1 = gmsh.model.occ.addPoint(0, height / 2, 0)
pp2 = gmsh.model.occ.addPoint(base / 4, height / 2, 0)
pp3 = gmsh.model.occ.addPoint(base / 8, height / 4, 0)

pl1 = gmsh.model.occ.addLine(pp1, pp2)
pl2 = gmsh.model.occ.addSpline([pp1, pp3, pp2])


gmsh.model.occ.synchronize()

In [315]:
FEM.plotOnPath(problem, pl1, sy, 100, name="σred", visible=false);

In [316]:
#gmsh.option.setNumber("PostProcessing.Binary", 1)
#gmsh.option.setNumber("PostProcessing.SaveMesh", 1)

#gmsh.write("rect.msh")
#gmsh.write("rect.opt")
#gmsh.view.write(4, "rect.pos")

In [317]:
gmsh.fltk.run()

-------------------------------------------------------
Version       : 4.12.0
License       : GNU General Public License
Build OS      : Linux64-sdk
Build date    : 20231221
Build host    : gmsh.info
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blas[petsc] Blossom Cgns DIntegration Dlopen DomHex Eigen[contrib] Fltk Gmm[contrib] Hxt Jpeg Kbipack Lapack[petsc] LinuxJoystick MathEx[contrib] Med Mesh Metis[contrib] Mmg Mpeg Netgen ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom PETSc Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR Voro++[contrib] WinslowUntangler Zlib
FLTK version  : 1.4.0
PETSc version : 3.14.4 (real arithmtic)
OCC version   : 7.7.2
MED version   : 4.1.0
Packaged by   : geuzaine
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------


In [318]:
gmsh.finalize()